### Read the data from the input Excel file

# Nouvelle section

In [1]:
import pandas as pd
inputFileName = "MineScheduling_smallDataSet.xlsx"
paramDF = pd.read_excel(inputFileName, "Param", skiprows=0)
jobDF = pd.read_excel(inputFileName, "Job", skiprows=0)
jobMachineDF = pd.read_excel(inputFileName, "JobMachine", skiprows=0)
switchingDF = pd.read_excel(inputFileName, "Switching", skiprows=0)

### Set the data into dict + compute maxDuration

In [2]:
horizonDuration = paramDF['Duration (day)'][0]

duration = dict()  #duration[vJob][vMachine]
maxDuration = 0.0
for iJob,vJob in enumerate(jobMachineDF['Job']):
    vMachine = jobMachineDF['Machine'][iJob]
    if not vJob in duration : duration[vJob] = dict() 
    duration[vJob][vMachine] = jobMachineDF['Duration'][iJob]
    if duration[vJob][vMachine] > maxDuration : maxDuration = duration[vJob][vMachine]

### Create your linear program

In [3]:
!pip install pulp

In [4]:
from pulp import *
prob = LpProblem("MineSchedulingProblem", LpMaximize)  

### Add variables

In [5]:
 #allocationVar[vJob][vMachine] & isFirst[vJob][vMachine]
allocationVar = dict()
isFirstVar = dict()
for iJob,vJob in enumerate(jobMachineDF['Job']):
    vMachine = jobMachineDF['Machine'][iJob]
    if not vJob in allocationVar : allocationVar[vJob] = dict() 
    if not vMachine in isFirstVar : isFirstVar[vMachine] = dict() 
    allocationVar[vJob][vMachine] = LpVariable("Allocation[%s][%s]"%(vJob,vMachine), cat='Binary')
    isFirstVar[vMachine][vJob] = LpVariable("isFirst[%s][%s]"%(vMachine, vJob), cat='Binary') 
    
#startVar[vJob] and isOver[vJob]
startVar = dict()
isOverVar = dict()
for vJob in jobDF['Id'] :
    startVar[vJob] = LpVariable("Start[%s]"%(vJob), lowBound=0, upBound=horizonDuration, cat='Continuous')
    isOverVar[vJob] = LpVariable("IsOver[%s]"%(vJob), lowBound=0, cat='Binary')

    
#isSuccessor[vMachine][vJob1-vJob2] = 1 if vJob2 is the direct successor of vJob1
isSuccessorVar = dict()  
for iMachine,vMachine in enumerate(switchingDF['Machine']):
    vJob1 = switchingDF['Job1'][iMachine]
    vJob2 = switchingDF['Job2'][iMachine]
    if not vMachine in isSuccessorVar : isSuccessorVar[vMachine] = dict()
    if not vJob1 in isSuccessorVar[vMachine] : isSuccessorVar[vMachine][vJob1] = dict() 
    if not vJob2 in isSuccessorVar[vMachine] : isSuccessorVar[vMachine][vJob2] = dict() 
    isSuccessorVar[vMachine][vJob1][vJob2] = LpVariable("IsSuccessor[%s][%s][%s]"%(vMachine, vJob1, vJob2), cat='Binary')
    isSuccessorVar[vMachine][vJob2][vJob1] = LpVariable("IsSuccessor[%s][%s][%s]"%(vMachine, vJob2, vJob1), cat='Binary')    
                        

### Add objective

In [6]:
#Define the production dictionary
jobProduction = dict(zip(jobDF.Id, jobDF.Production)) 
jobVolume = dict(zip(jobDF.Id, jobDF.Volume))

In [7]:
obj = LpAffineExpression()

obj += lpSum([isOverVar[j] * jobVolume[j] for j in jobDF['Id']])

prob += obj

### Add constraints

In [8]:
#Max one allocation per job
#Define a dictionary : key = Job, value = Associated machines
MachineJob_dict = dict(jobMachineDF.groupby('Job')['Machine'].apply(list))
for vJob in jobDF['Id']:   
    prob += lpSum(allocationVar[vJob][vMachine] for vMachine in MachineJob_dict[vJob] ) <= 1

In [9]:
#Max one "isFirst" job per machine
JobMachine_dict = dict(jobMachineDF.groupby('Machine')['Job'].apply(list))
for vMachine in paramDF['Machine']:
    prob += lpSum(isFirstVar[vMachine][vJob] for vJob in JobMachine_dict[vMachine] ) <= 1

In [10]:
BigM =1000

In [11]:
from os import kill
#Constraint stating the "start" variable
#Start[j’] >= Start[j] + duration[j][m] + switchingTime[m][j][j’]  - BigM * (1 - isSuccessor[m][j][j’])
#Start[j'] - Start[j] - BigM * isSuccessor[m][j][j'] >= duration[j][m] + switchingTime[m][j][j’] - BigM

for iMachine,vMachine in enumerate(switchingDF['Machine']):
   vJob1 = switchingDF['Job1'][iMachine]
   vJob2 = switchingDF['Job2'][iMachine]
   prob += startVar[vJob1]+duration[vJob1][vMachine]+switchingDF['SwitchingTime (day)'][iMachine]-BigM*(1-isSuccessorVar[vMachine][vJob1][vJob2]) <=startVar[vJob2] , "SwitchingConst[%s][%s][%s]"%(iMachine,vJob1,vJob2)


In [12]:
#Constraint stating the "isOver" variable
#start[j] + Sum[m in isPossible[j] ] duration[j][m] * allocation[j][m] <= durationHorizon + BigM * (1 - isOver[j])
#start[j] + Sum[m in isPossible[j] ] duration[j][m] * allocation[j][m] + BigM * isOver[j] <= durationHorizon + BigM
for vJob in jobDF['Id']:
  prob += startVar[vJob]+ lpSum(duration[vJob][vMachine] * allocationVar[vJob][vMachine] for vMachine in MachineJob_dict[vJob])  <= horizonDuration + BigM * (1 - isOverVar[vJob])
  

In [13]:
#Constraint stating that to be allocated, a job shall either be the first, or be the successor of another job
# allocationVar[vJob][vMachine] <= isFirst[vJob][vMachine] + sum[otherJobs oJob] isSuccessor[vMachine][oJob][vJob]

for iJob,vJob in enumerate(jobDF['Id']):
    for vMachine in MachineJob_dict[vJob]:
        otherjobs = switchingDF[switchingDF['Job2'] == vJob ]
        otherjobs = otherjobs[otherjobs['Machine'] == vMachine]['Job1']
        prob += allocationVar[vJob][vMachine] <= isFirstVar[vMachine][vJob] +lpSum(isSuccessorVar[vMachine][oJob][vJob] for oJob in otherjobs)

In [14]:
#Constraint stating that if isOver = 1, then at least one allocation = 1
for vJob in jobDF['Id']:
    prob+=  isOverVar[vJob] <= lpSum(allocationVar[vJob][vMachine] for vMachine in MachineJob_dict[vJob] )    

In [15]:
#Max one successor per job
for iJob,vJob in enumerate(jobMachineDF['Job']):
      prob += lpSum ( isSuccessorVar[vMachine][vJob][oJob] for vMachine in MachineJob_dict[vJob] for oJob in isSuccessorVar[vMachine][vJob] ) <= 1


In [16]:
#other constraint

In [17]:
machine = list(paramDF['Machine'])
machine = paramDF[['Machine']]
machine_names = machine.Machine.unique()
Jobmachine_dict=dict(jobMachineDF.groupby('Machine')['Job'].apply(list))   

In [18]:
#Constraint stating that IsSuccessor[m][j1][j2] = 1 --> allocation[j1][m] = 1
for m in machine_names:
    for j in jobDF['Id']:
        if Jobmachine_dict[m] != j:
               otherjobmachine=Jobmachine_dict[m]
        continue
        for oj in otherjobmachine:
                prob += isSuccessorVar[m][j][oj] <= allocationVar[vJob][vMachine]

In [19]:
#9Constraint stating that if start > 0, then then at least one allocation = 1
for vJob in jobDF['Id']:
    prob += startVar[vJob] <= BigM * lpSum(allocationVar[vJob][vMachine] for vMachine in MachineJob_dict[vJob] )

In [20]:
#If Sum allocation = 1, then start >= 1.0

for vJob in jobDF['Id']:
    prob +=  lpSum(allocationVar[vJob][vMachine] for vMachine in MachineJob_dict[vJob]) <= startVar[vJob] 

In [21]:
## Constraint imposing predecessors : isOver[j] = 1  isOver[vPred] = 1  
jPred_dict=dict(zip(jobDF['Id'], jobDF['PredecessorId'])) 
for j in jobDF['Id']:
    if jPred_dict[j] in jobDF['Id']:
        prob +=  isOverVar[j] <= isOverVar[jPred_dict[j]]
    continue

In [22]:
 # Constraint imposing predecessors :   j shall start only once vPred has finished
for j in jobDF['Id']:
    jPred=jPred_dict[j]
    if jPred not in jobDF['Id']:
        continue
    prob += StartVar[jPred] + lpSum(duration[jPred][m] * allocationVar[jPred][m] for m in MachineJ_dict[jPred])<= StartVar[j]

### Solve and display results

In [23]:
prob.writeLP("mineSchedulingProblem.lp", writeSOS=1, mip=1)
prob.solve()
print("Status:", LpStatus[prob.status])
print ("Objective = ", value(prob.objective))
varsDict = {}
for v in prob.variables():
    varsDict[v.name] = v.varValue
    if "IsOver" in v.name or "Start" in v.name or "Allocation" in v.name or "IsSuccessor" in v.name:
        if v.varValue != 0.0 : print(v.name, "=", v.varValue)    

Status: Optimal
Objective =  2900.0
Allocation_Job11__Bull_ = 1.0
Allocation_Job12__Bull_ = 1.0
Allocation_Job13__SmallDragline_ = 1.0
Allocation_Job14__BigDragline_ = 1.0
Allocation_Job15__BigDragline_ = 1.0
Allocation_Job16__BigDragline_ = 1.0
Allocation_Job17__BigDragline_ = 1.0
Allocation_Job18__BigDragline_ = 1.0
IsOver_Job11_ = 1.0
IsOver_Job12_ = 1.0
IsOver_Job13_ = 1.0
IsOver_Job14_ = 1.0
IsOver_Job15_ = 1.0
IsOver_Job16_ = 1.0
IsOver_Job17_ = 1.0
IsSuccessor_BigDragline__Job10__Job14_ = 1.0
IsSuccessor_BigDragline__Job5__Job18_ = 1.0
IsSuccessor_BigDragline__Job8__Job16_ = 1.0
IsSuccessor_BigDragline__Job9__Job15_ = 1.0
IsSuccessor_Bull__Job1__Job11_ = 1.0
Start_Job11_ = 23.904762
Start_Job12_ = 1.0
Start_Job13_ = 1.0
Start_Job14_ = 28.933333
Start_Job15_ = 34.0
Start_Job16_ = 27.454545
Start_Job17_ = 1.0
Start_Job18_ = 50.0
